# ICMP Testing

https://keith.github.io/xcode-man-pages/icmp.4.html

Ping sockets are defined in [net/ipv4/af_inet.c](https://github.com/torvalds/linux/blob/ec2df4364666a96e7868b7257bc7235bae263dcb/net/ipv4/af_inet.c#L1172-L1178) and implemented in [net/ipv4/ping.c](https://github.com/torvalds/linux/blob/ec2df4364666a96e7868b7257bc7235bae263dcb/net/ipv4/ping.c#L990-L1010).  Likewise, IPv6 ping sockets are defined in [net/ipv6/af_inet6.c](https://github.com/torvalds/linux/blob/ec2df4364666a96e7868b7257bc7235bae263dcb/net/ipv6/af_inet6.c#L1107) and implemented in [net/ipv6/ping.c](https://github.com/torvalds/linux/blob/ec2df4364666a96e7868b7257bc7235bae263dcb/net/ipv6/ping.c#L197-L217).  They're effectively a cut-down version of raw sockets with added controls.

```c
socket(AF_INET, SOCK_DGRAM, IPPROTO_ICMP)
```

Generated using this:

```bash
zcat /usr/share/man/man7/ip.7.gz | pandoc --from man --to html --wrap=preserve > ip.html
```

You can use the following options:

[...](./socket-stuff.md)


## IPv4 Header

<table>
    <tr>
        <th style="text-align: center; border: 1px solid black">Offset</th>
        <th style="text-align: center; border: 1px solid black" colspan="8">0</th>
        <th style="text-align: center; border: 1px solid black" colspan="8">1</th>
        <th style="text-align: center; border: 1px solid black" colspan="8">2</th>
        <th style="text-align: center; border: 1px solid black" colspan="8">3</th>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">Octet</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">0</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">1</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">2</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">3</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">4</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">5</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">6</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">7</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">8</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">9</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">10</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">11</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">12</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">13</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">14</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">15</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">16</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">17</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">18</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">19</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">20</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">21</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">22</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">23</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">24</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">25</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">26</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">27</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">28</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">29</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">30</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">31</th>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">0</th>
        <td style="text-align: center; border: 1px solid black" colspan="4">Version (4)</td>
        <td style="text-align: center; border: 1px solid black" colspan="4">IHL</td>
        <td style="text-align: center; border: 1px solid black" colspan="6">DSCP</td>
        <td style="text-align: center; border: 1px solid black" colspan="2">ECN</td>
        <td style="text-align: center; border: 1px solid black" colspan="16">Total Length</td>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">4</th>
        <td style="text-align: center; border: 1px solid black" colspan="16">Identification</td>
        <td style="text-align: center; border: 1px solid black" colspan="3">Flags</td>
        <td style="text-align: center; border: 1px solid black" colspan="13">Fragment Offset</td>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">8</th>
        <td style="text-align: center; border: 1px solid black" colspan="8">Time to Live</td>
        <td style="text-align: center; border: 1px solid black" colspan="8">Protocol</td>
        <td style="text-align: center; border: 1px solid black" colspan="16">Header Checksum</td>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">12</th>
        <td style="text-align: center; border: 1px solid black" colspan="32">Source address</td>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">16</th>
        <td style="text-align: center; border: 1px solid black" colspan="32">Destination address</td>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">20</th>
        <td style="text-align: center; border: 1px solid black" colspan="32" rowspan="3">Options</td>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">...</th>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">56</th>
    </tr>

</table>


# ICMP Echo Request

<table>
    <tr>
        <th style="text-align: center; border: 1px solid black">Offset</th>
        <th style="text-align: center; border: 1px solid black" colspan="8">0</th>
        <th style="text-align: center; border: 1px solid black" colspan="8">1</th>
        <th style="text-align: center; border: 1px solid black" colspan="8">2</th>
        <th style="text-align: center; border: 1px solid black" colspan="8">3</th>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">Octet</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">0</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">1</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">2</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">3</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">4</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">5</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">6</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">7</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">8</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">9</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">10</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">11</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">12</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">13</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">14</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">15</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">16</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">17</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">18</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">19</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">20</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">21</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">22</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">23</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">24</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">25</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">26</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">27</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">28</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">29</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">30</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">31</th>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">0</th>
        <td style="text-align: center; border: 1px solid black" colspan="8">Type</td>
        <td style="text-align: center; border: 1px solid black" colspan="8">Code</td>
        <td style="text-align: center; border: 1px solid black" colspan="16">Checksum</td>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">4</th>
        <td style="text-align: center; border: 1px solid black" colspan="16">Identifier</td>
        <td style="text-align: center; border: 1px solid black" colspan="16">Sequence Number</td>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">8</th>
        <td style="text-align: center; border: 1px solid black" colspan="32" rowspan="2">Optional Data</td>
    </tr>
    <tr>
        <th style="text-align: center; border: 1px solid black">...</th>
    </tr>
</table>

This can be created using the following code:

> ```py
> import struct
> 
> header = struct.pack(
>     "!BBHHH",   # Format
>     1,          # Field value: Type
>     2,          # Field value: Code
>     3,          # Field value: Checksum
>     4,          # Field value: Identifier
>     5           # Field value: Sequence Number
> )
> ```

In [9]:
import struct

header = struct.pack(
    "!BBHHH",   # Format
    1,          # Field value: Type
    2,          # Field value: Code
    3,          # Field value: Checksum
    4,          # Field value: Identifier
    5           # Field value: Sequence Number
)

b'\x01\x02\x00\x03\x00\x04\x00\x05'

In [15]:
for x in range(0,32):
    pass
    # print(f"""        <th style="text-align: center; border: 1px solid black; padding-left: 2px; padding-right: 2px">{x}</th>""")
    print(f"""        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">{x}</th>""")


        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">0</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">1</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">2</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">3</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">4</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">5</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">6</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">7</th>
        <th style="text-align: center; border: 1px solid black; padding-left: 4px; padding-right: 4px">8</th>
        <t

In [2]:
SO_BINDTODEVICE = 25
SIOCGIFADDR = 0x8915

import socket

# Create 
